In [1]:
import pandas as pd
import spacy
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import fasttext.util
from sklearn.model_selection import train_test_split

# Pipeline für Textklassifikation mit fasttext

## Datenvorbereitung

In [2]:
# load data
sample = pd.read_csv("data/labeled_unprocessed_sample_data.csv", sep=";")

In [3]:
# remove possible missing values and duplicates
sample = sample.dropna()
sample = sample.drop_duplicates(subset="answer")

In [4]:
nlp = spacy.load("de_core_news_sm")

def text_lemmatization(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_punct]
    return " ".join(lemmas)

In [5]:
def text_preprocessing(text):
    # remove links, punctuation, special letters
    text = re.sub(r"[^a-zA-öZÖäÄüÜß]|\bhttps?://\S*|&\w+;|[\.,]", " ", text)
    
    # replace single characters
    text = re.sub(r" [a-zA-Z] ", " ", text)
    
    # remove additional whitespaces
    text = re.sub(r"\s+", " ", text)
    
    # lemmatize texts
    text = text_lemmatization(text)

    # lower text
    text = text.lower()
    
    # tokenization of words
    text = text.split()
    
    # remove stopwords
    german_stopwords = set(stopwords.words("german"))
    text = [w for w in text if w not in german_stopwords]
    
    # return joined text
    return " ".join(text)

In [6]:
sample["clean_answer"] = sample["answer"].apply(text_preprocessing)

In [7]:
# show most frequent words
vectorizer = CountVectorizer(
)
dtm = vectorizer.fit_transform(sample["clean_answer"])

frequencies = dtm.sum(axis=0).tolist()[0]

df_freq = pd.DataFrame(
    dict(frequencies=frequencies,
         index=vectorizer.get_feature_names_out()
    )
)

df_freq.sort_values("frequencies", ascending=False).head(n=20)

,frequencies,index
9221,2012,geehrt
8766,1718,freundlich
24838,1712,vieler
11383,1698,herr
8594,1579,frage
4977,1279,dank
20807,971,sollen
15637,942,mensch
9138,900,geben
10810,898,gut


In [8]:
# remove stopwords with refined list and after lemmatization to catch 
def remove_individual_stopwords(text):
    individual_stopwords = [
        "geehrt",
        "frau",
        "vieler",
        "dank",
        "herr",
        "danke",
        "anfrage",
        "frage",
        "nachricht",
        "freundlich",
        "sollen",
        "müssen",
        "mehr",
        "grüße",
        "daher",
        "immer",
        "dafür",
        "frage"
    ]
    text = text.split()
    text = [w for w in text if w not in individual_stopwords]
    
    return " ".join(text)

In [9]:
sample["clean_answer"] = sample["clean_answer"].apply(remove_individual_stopwords)

In [10]:
# prepare data for processing by fasttext
sample["answer_encoded"] = sample["answer_encoded"].apply(lambda x: x.replace(" ", "_"))
sample["answer_encoded"] = sample["answer_encoded"].apply(lambda x: "__label__" + x)
sample["answer_encoding_combined"] = sample["answer_encoded"]+ " " + sample["clean_answer"]

In [11]:
# seperate sample data into training, test and validation sets (80/10/10)
training_sample, temp_df = train_test_split(sample, test_size=0.2, random_state=42)
testing_sample, validation_sample = train_test_split(temp_df, test_size=0.5, random_state=42)

In [12]:
# export datasets
training_sample["answer_encoding_combined"].to_csv("data/training_data.csv", index=False, header=False, sep=";")
testing_sample["answer_encoding_combined"].to_csv("data/testing_data.csv", index=False, header=False, sep=";")
validation_sample["answer_encoding_combined"].to_csv("data/validation_data.csv", index=False, header=False, sep=";")

## Modelltraining

### Ohne optimierte Hyperparamater

In [13]:
# train model without optimizing the hyperparameter
ft_model = fasttext.train_supervised(input="data/training_data.csv")

In [14]:
# test model
ft_model.test("data/testing_data.csv")

(228, 0.6578947368421053, 0.6578947368421053)

### Mit optimierten Hyperparametern

In [15]:
# train model with optimizing the hyperparameter
ft_model_optimized = fasttext.train_supervised(
    input="data/training_data.csv", 
    autotuneValidationFile="data/validation_data.csv"
    )

In [16]:
# test model
ft_model_optimized.test("data/testing_data.csv")

(228, 0.6754385964912281, 0.6754385964912281)

#### Testergebnisse für Label "answer"

In [17]:
# test model for label "answer"
ft_model_optimized.test_label("data/testing_data.csv")["__label__answer"]

{'precision': 0.678391959798995,
 'recall': 0.9310344827586207,
 'f1score': 0.7848837209302325}

#### Testergebnisse für Label "evasive answer"

In [18]:
# test model for label "evasive answer"
ft_model_optimized.test_label("data/testing_data.csv")["__label__evasive_answer"]

{'precision': 0.6551724137931034,
 'recall': 0.2289156626506024,
 'f1score': 0.3392857142857143}

In [19]:
# save trained model
ft_model_optimized.save_model("model/classification.bin")

## Modellanwendung

### Laden und Vorverarbeiten der Daten

In [24]:
# load unprocessed data without sample
data = pd.read_csv("data/data_without_sample.csv", sep=";")

In [27]:
# drop na and duplicates in answers
data = data.drop_duplicates(subset=["answer", "question_text"])
data = data.dropna(subset=["answer"])

In [30]:
# apply text cleaning
data["clean_answer"] = data["answer"].apply(text_preprocessing)
data["clean_answer"] = data["clean_answer"].apply(remove_individual_stopwords)


### Automatisches Labeln der Antworten

In [31]:
# extract answers from colums
text = data["clean_answer"].tolist()

In [ ]:
# predict label and probability of label for each answer
labels, probalities = ft_model_optimized.predict(text, k=1)

### Nachbearbeiten der Labels und Probabilities für bessere Lesbarkeit

In [41]:
def flatten_list(list):
    flattened_list = []
    for item in list:
        for i in item:
            flattened_list.append(i)
    return flattened_list

flattened_labels = flatten_list(labels)
flattened_probabilities = flatten_list(probalities)

In [42]:
# append labels and probabilities to dataframe
data["label"] = flattened_labels
data["probability"] = flattened_probabilities

In [ ]:
data["label"] = data["label"].apply(lambda x: x.replace("__label__", ""))

In [44]:
data.head()

,party,last_name,gender,answer,topic,question_text,question_teaser,answer_encoded,clean_answer,label,probability
0,FDP,Buschmann,m,"Sehr geehrter Herr G., haben Sie vielen Dank f...",Bildung und Erziehung,Gerichte entscheiden fast ausschließlich gegen...,Wie wollen Sie das Wechselmodell stärker förd...,NaN,brauchen familienrecht höhe zeit zählen insbes...,answer,0.796268
1,FDP,Buschmann,m,"Sehr geehrter Herr S., haben Sie vielen Dank f...",Recht,"Sehr geehrter Herr Buschmann,ich beziehe mich ...",Wieso wird die tatsächliche Postlaufzeit um 2...,NaN,neufassung abs postg heben durchschnittlich br...,answer,0.742809
2,FDP,Buschmann,m,"Sehr geehrter Herr E.,haben Sie vielen Dank fü...",Soziale Sicherung,Zur vorherigen Frage eine Berichtigung: Das Sc...,Das Schonvermögen darf man selbstverständlich...,NaN,klar wer freiwillig arbeiten netto brutto wer ...,answer,0.798249
3,FDP,Buschmann,m,"Sehr geehrter Herr D.,haben Sie vielen Dank fü...",Lobbyismus & Transparenz,Sehr geehrter Herr Buschmann.Obwohl höchsten R...,Das Weisungsrecht gegenüber der Exekutive: Ob...,NaN,klar unabhängigkeit justiz wesentlich baustein...,answer,0.733446
4,FDP,Buschmann,m,"Sehr geehrte Frau S.,haben Sie vielen Dank für...",Landwirtschaft und Ernährung,NaN,"Anbindehaltung, Ferkelstände , wann wird dies...",NaN,frei demokrat treten verantwortungsvoll haltun...,answer,0.642618
